In [103]:
import numpy as np
%load_ext memory_profiler

def get_pos_neg_fp_lists(dataset_name):
    pos_fp_list = glob.glob(os.path.join("..", "data", "02_intermediate", dataset_name, "*", "1", "*.npz"))
    neg_fp_list = glob.glob(os.path.join("..", "data", "02_intermediate", dataset_name, "*", "0", "*.npz"))
    return pos_fp_list, neg_fp_list

def samples_from_fp_list(fp_list):
    return [np.load(f) for f in fp_list]

def read_tpa123_from_npz(npz_sample, dtype=np.float32):
    ids = ('121', '122', '123')
    return [np.expand_dims(npz_sample['array_ID{}'.format(id)],axis=-1).astype(dtype) for id in ids]

DATASET_NAME = "/media/igor/DATA/PIPELINE/20200508_aligned_labeled_npz_f100_fs0"

import glob
import os

from sys import getsizeof

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


a is around 150 MB

In [31]:
%memit a = np.random.rand(int(20e6))

peak memory: 369.00 MiB, increment: 304.57 MiB


In [32]:
a = np.random.rand(int(20e6)).reshape(-1,20,20)

In [33]:
%memit a.flatten()

peak memory: 359.48 MiB, increment: 142.46 MiB


In [34]:
%memit np.mean(a)

peak memory: 217.05 MiB, increment: 0.00 MiB


In [35]:
%memit np.std(a)

peak memory: 369.62 MiB, increment: 152.57 MiB


In [36]:
%%memit
a = np.random.rand(int(20e6))
std = a.std()
mean = a.std()

peak memory: 369.51 MiB, increment: 152.46 MiB


In [37]:
del(a, mean, std)

In [119]:
import psutil
mem = psutil.virtual_memory()
available_MiB = mem.available >> 20
print(available_MiB)

8837


In [121]:
%%memit
import psutil
mem = psutil.virtual_memory()
available_MiB = mem.available >> 20
raw_data_limit_MiB = available_MiB//3

pos_fp_list, neg_fp_list = get_pos_neg_fp_lists(DATASET_NAME)
pos_samples = samples_from_fp_list(pos_fp_list)
neg_samples = samples_from_fp_list(neg_fp_list)

data = np.array([0], dtype=np.float16)
for sample in pos_samples + neg_samples:
    tpa1, tpa2, tpa3 = read_tpa123_from_npz(sample, dtype=np.float16)
    tpas = np.concatenate([tpa1, tpa2, tpa3]).flatten()
    data = np.append(data, tpas)
    mem_used_MiB = (getsizeof(data) >> 20) * 2 # this will be in float32 later
    if mem_used_MiB >= raw_data_limit_MiB:
        break
data = data.astype(np.float32)
std = data.std()
mean = data.mean()

peak memory: 1618.85 MiB, increment: 1205.27 MiB


In [116]:
print(raw_data_limit_MiB)

3044


In [117]:
os.platform()

AttributeError: module 'os' has no attribute 'platform'